In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import os
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Fungsi untuk membuka browser dan melakukan scraping
def scrape_images(input_csv, output_csv):
    try:
        # Mengatur opsi untuk Chrome (headless mode untuk menjalankan di background)
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Path ke ChromeDriver
        driver_path = r"C:\Users\Asus\Documents\PROJECT\chromedriver-win64\chromedriver.exe"
        driver = webdriver.Chrome(service=Service(driver_path), options=chrome_options)

        # Membaca CSV input yang berisi daftar obat
        df = pd.read_csv(input_csv, sep=';')

        # Membaca file output jika sudah ada
        if os.path.exists(output_csv):
            results_df = pd.read_csv(output_csv)
            existing_names = results_df['name'].tolist()
        else:
            existing_names = []

        # Menyiapkan list untuk menyimpan hasil scraping
        results = []

        for index, row in df.iterrows():
            # Mengambil nama obat dari csv dan memprosesnya
            obat = row['name']
            if obat in existing_names:
                print(f"{obat} sudah ada di file, skip.")
                continue

            obat_preprocessed = obat.lower().replace("/", '-').replace("'", '-').replace('%', '').replace('.', '-').replace(' ', '-')
            print('Running scraping untuk:', obat_preprocessed)

            # Akses URL berdasarkan nama obat yang sudah diproses
            url = f"https://www.halodoc.com/obat-dan-vitamin/{obat_preprocessed}"
            driver.get(url)
            time.sleep(3)

            try:
                # Menunggu dan mencari elemen gambar produk
                image_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//img[@class='product-image']"))
                )

                # Mengambil URL gambar dari elemen img
                image_url = image_element.get_attribute('src')

                # Menambahkan data ke hasil
                results.append({'name': obat, 'Image URL': image_url})

            except Exception as e:
                # Jika gambar tidak ditemukan atau terjadi kesalahan, lanjutkan ke obat berikutnya
                print(f"Error scraping {obat}: {e}")
                continue

            # Menyimpan hasil sementara setelah setiap iterasi untuk menghindari kehilangan data
            if results:
                results_df = pd.DataFrame(results)
                if os.path.exists(output_csv):
                    # Jika file output sudah ada, gabungkan dengan hasil baru
                    results_df = pd.concat([results_df, pd.read_csv(output_csv)], ignore_index=True)
                results_df.to_csv(output_csv, index=False)
                print(f"Data sementara disimpan di {output_csv}")

        # Jika masih ada hasil yang belum disimpan
        if results:
            results_df = pd.DataFrame(results)
            if os.path.exists(output_csv):
                # Jika file output sudah ada, gabungkan dengan hasil baru
                results_df = pd.concat([results_df, pd.read_csv(output_csv)], ignore_index=True)
            results_df.to_csv(output_csv, index=False)
            print(f"Scraping selesai, hasil final disimpan di {output_csv}")

        else:
            print("Tidak ada hasil baru untuk disimpan.")

    except Exception as e:
        # Jika ada error dalam proses utama, simpan data yang sudah ada
        print(f"Terjadi error: {e}")
        if results:
            results_df = pd.DataFrame(results)
            if os.path.exists(output_csv):
                # Jika file output sudah ada, gabungkan dengan hasil baru
                results_df = pd.concat([results_df, pd.read_csv(output_csv)], ignore_index=True)
            results_df.to_csv(output_csv, index=False)
            print(f"Data yang sudah ada berhasil disimpan di {output_csv}")
        else:
            print("Tidak ada hasil yang berhasil disimpan.")

    finally:
        # Menutup browser di akhir eksekusi, bahkan jika terjadi error
        driver.quit()

# Menjalankan fungsi scraping
input_csv = 'data_halodoc_ordered.csv'
output_csv = 'hasil_scraping.csv'
scrape_images(input_csv, output_csv)

Kloderma 0.05% Salep 5 g sudah ada di file, skip.
Lotasbat 0.05% Cream 10 g sudah ada di file, skip.
Fibrefirst 15 Sachet sudah ada di file, skip.
Fibrefirst 7 Sachet sudah ada di file, skip.
Fibrefirst 15 Sachet sudah ada di file, skip.
Fibrefirst 7 Sachet sudah ada di file, skip.
My Baby Minyak Telon Plus Lavender 60 ml sudah ada di file, skip.
Viostin 5 Kaplet sudah ada di file, skip.
Eskulin Spray Cologne Japan Spring Melody 110 ml sudah ada di file, skip.
Sido Muncul Libidione 30 Kapsul sudah ada di file, skip.
Lip Ice Color Naughty Red sudah ada di file, skip.
Aza 20 gel 30 g sudah ada di file, skip.
Bodrexin Herbal Batuk Sachet 7.5 ml sudah ada di file, skip.
X-Gra 150 mg 10 Kapsul sudah ada di file, skip.
Bodrexin Herbal Batuk Sachet 7.5 ml sudah ada di file, skip.
L-Bio 15 Kapsul sudah ada di file, skip.
Aza 20 Cream 10 g sudah ada di file, skip.
Borobudur Asi 30 Kapsul sudah ada di file, skip.
Erha Follicore Herbal 60 Kapsul sudah ada di file, skip.
X-Gra 150 mg 10 Kapsul sud

KeyboardInterrupt: 

In [1]:
import pandas as pd

# Membaca file CSV
file_path = 'hasil_scraping.csv'
df = pd.read_csv(file_path)

# Menghapus duplikat berdasarkan kolom 'name'
df_cleaned = df.drop_duplicates(subset=['name'], keep='first')

# Mencari baris dengan URL yang hilang (kosong atau NaN)
missing_urls = df_cleaned[df_cleaned['Image URL'].isna()]

# Menyimpan data yang sudah dibersihkan tanpa duplikat ke file baru
df_cleaned.to_csv('data_img_halodoc.csv', index=False)

# Menyimpan daftar nama obat dengan URL yang hilang ke file baru
print(missing_urls)

# Menampilkan jumlah data yang hilang URL-nya
print(f"Jumlah obat yang tidak memiliki URL gambar: {missing_urls.shape[0]}")

Empty DataFrame
Columns: [name, Image URL]
Index: []
Jumlah obat yang tidak memiliki URL gambar: 0


In [2]:
# Load data dari kedua file CSV
data_halodoc = pd.read_csv('data_halodoc_ordered.csv', sep=';')
hasil_scraping = pd.read_csv('hasil_scraping.csv')

# Cek apakah kolom 'name' ada di kedua DataFrame
if 'name' in data_halodoc.columns and 'name' in hasil_scraping.columns:
    # Cek apakah nilai kolom 'name' di data_halodoc sudah ada di hasil_scraping
    unmatched_names = ~data_halodoc['name'].isin(hasil_scraping['name'])
    
    # Menampilkan hasil
    if unmatched_names.any():
        print("Beberapa nama obat di data_halodoc_ordered.csv tidak ada di hasil_scraping.csv.")
        # Menampilkan nama obat yang tidak ada di hasil_scraping
        print("Nama obat yang tidak ada di hasil_scraping.csv:")
        print(data_halodoc['name'][unmatched_names])
    else:
        print("Semua nama obat di data_halodoc_ordered.csv ada di hasil_scraping.csv.")
else:
    print("Kolom 'name' tidak ditemukan di salah satu atau kedua file.")

Beberapa nama obat di data_halodoc_ordered.csv tidak ada di hasil_scraping.csv.
Nama obat yang tidak ada di hasil_scraping.csv:
41                             Rohto Fever Patch 2 Lembar
49                             Rohto Fever Patch 2 Lembar
103     Intihealth Calcium Magnesium Zinc Vitamin D 30...
110                                 Astria 4 mg 12 Kapsul
124                                 Astria 4 mg 12 Kapsul
                              ...                        
4727                    Labore On The Go Minis II Package
4728               Erha Nutraceuticals Skincore 30 Kapsul
4729    Labore Sensitive Skin Care Biomerepair Topical...
4730    Labore Sensitive Skin Care Biomerepair Topical...
4731                   Value Pack Face Brightening Series
Name: name, Length: 3855, dtype: object


In [3]:
# Menemukan duplikasi berdasarkan kolom 'name'
duplicates = data_halodoc[data_halodoc.duplicated(subset=['name'], keep=False)]

# Menampilkan hasil duplikasi
if not duplicates.empty:
    print("Baris yang terduplikasi:")
    print(duplicates)
else:
    print("Tidak ada data yang terduplikasi.")

Baris yang terduplikasi:
                                                   name  \
2                                  Fibrefirst 15 Sachet   
3                                   Fibrefirst 7 Sachet   
4                                  Fibrefirst 15 Sachet   
5                                   Fibrefirst 7 Sachet   
12                  Bodrexin Herbal Batuk Sachet 7.5 ml   
...                                                 ...   
4716  SGM Eksplor 5 Plus Complinutri Madu Susu Bubuk...   
4717  SGM Eksplor 5 Plus Complinutri Madu Susu Bubuk...   
4718  SGM Eksplor 5 Plus Complinutri Madu Susu Bubuk...   
4722  La Roche Posay Spot Scan Acne Fighter Essentia...   
4723  La Roche Posay Spot Scan Acne Fighter Essentia...   

                      price  \
2                 Rp245.000   
3     Rp127.400 - Rp206.200   
4                 Rp245.000   
5     Rp127.400 - Rp206.200   
12        Rp1.500 - Rp2.500   
...                     ...   
4716    Rp36.800 - Rp51.700   
4717   Rp45.900 - 